Importando bibliotecas

In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument 

import pandas as pd
import numpy as np
import collections
import logging
import random

Configurando nível de logs

In [35]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Lendo arquivos de texto

In [69]:
rfzd_claims = pd.read_parquet('../dataset/refined/claims.parquet.snappy')

Dividindo entre conjunto de treino e teste

In [75]:
train_set = rfzd_claims.sample(frac=0.8, random_state=51)
test_set = rfzd_claims.drop(train_set.index)

train_set.reset_index(inplace=True, drop=True)
test_set.reset_index(inplace=True, drop=True)

Criando tag para armazenar marcar cada documento.

In [78]:
train_documents = []

for index, data in train_set.iterrows():
    train_documents.append(TaggedDocument(data['tokens_customer_claim'], [index]))

In [79]:
test_documents = test_set['tokens_customer_claim'].values.tolist()

Definindo modelo Doc2vec

In [81]:
model = Doc2Vec(
    vector_size=400, # tamanho do vetor de embedding
    epochs=8, # número de iterações no dataset inteiro para o treinamento
    min_count=5, # quantidade mínima de repetições palavras para entrarem no treinamento 
    sample=10**-5, # frequência teórica para ponderar palavras muito frequêntes
    hs=0, # utilizando amostragem negativa para acelarar o treinamento
    negative=20, # determina que 20 palavras aleatórias serão utilizadas para treinar a palavra predita
    dm=1, # usando modelo PV-DM
    dm_concat=1, # concatenando vetor de parágrafo com de palavras
    dbow_words=1, # treinando também vetor de palavras word2vec
    shrink_windows=True, # define a quantidade de palavras dinamicamente durante o treinamento
    workers=8, # numero de cores utilizados no paralelismo
    seed=51, # fixando a aleatoriada para deixar o modelo reprodutível
)

2024-09-18 17:41:05,114 : INFO : using concatenative 4400-dimensional layer1
2024-09-18 17:41:05,120 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/c,d400,n20,w5,mc5,s1e-05,t8>', 'datetime': '2024-09-18T17:41:05.120029', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}


Criando vocabulário

In [82]:
model.build_vocab(train_documents)

2024-09-18 17:41:10,332 : INFO : collecting all words and their counts
2024-09-18 17:41:10,333 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2024-09-18 17:41:10,523 : INFO : PROGRESS: at example #10000, processed 952590 words (5038060 words/s), 24988 word types, 0 tags
2024-09-18 17:41:10,702 : INFO : PROGRESS: at example #20000, processed 1905647 words (5355417 words/s), 34637 word types, 0 tags
2024-09-18 17:41:10,879 : INFO : PROGRESS: at example #30000, processed 2863831 words (5469280 words/s), 42004 word types, 0 tags
2024-09-18 17:41:11,056 : INFO : PROGRESS: at example #40000, processed 3827504 words (5481977 words/s), 48323 word types, 0 tags
2024-09-18 17:41:11,260 : INFO : PROGRESS: at example #50000, processed 4776326 words (4661947 words/s), 53574 word types, 0 tags
2024-09-18 17:41:11,451 : INFO : PROGRESS: at example #60000, processed 5753839 words (5155520 words/s), 58453 word types, 0 tags
2024-09-18 17:41:11,653 : INFO : PROGRES

In [83]:
model.train(
    train_documents,
    total_examples=model.corpus_count,
    epochs=32
)

2024-09-18 17:42:08,326 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 38563 vocabulary and 4400 features, using sg=0 hs=0 sample=1e-05 negative=20 window=5 shrink_windows=True', 'datetime': '2024-09-18T17:42:08.326439', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'train'}
2024-09-18 17:42:09,573 : INFO : EPOCH 0 - PROGRESS: at 0.31% examples, 16497 words/s, in_qsize 15, out_qsize 0
2024-09-18 17:42:10,611 : INFO : EPOCH 0 - PROGRESS: at 0.78% examples, 22929 words/s, in_qsize 15, out_qsize 0
2024-09-18 17:42:11,612 : INFO : EPOCH 0 - PROGRESS: at 1.26% examples, 25629 words/s, in_qsize 15, out_qsize 0
2024-09-18 17:42:12,737 : INFO : EPOCH 0 - PROGRESS: at 1.80% examples, 27419 words/s, in_qsize 15, out_qsize 0
2024-09-18 17:42:13,873 : INFO : EPOCH 0 - PROGRESS: at 2.36% examples, 28359 words/s, in_qsize 15, out_qsize 0
2024-09-18 17:42:14,976 : INFO : EP

In [84]:
vector = model.infer_vector('gostaria de fazer uma reclamação'.split(' '))
print(vector)

[ 0.00290261 -0.04821131  0.00093583  0.03354463 -0.01139435 -0.01557009
 -0.04210756 -0.03678525  0.00677327 -0.02304613 -0.00405972  0.01058661
 -0.02079258  0.03808933 -0.00183608 -0.01769445  0.00540106 -0.00329736
 -0.03533761  0.00526071 -0.00044301 -0.01421114 -0.0033552   0.02786181
  0.02508982  0.0038612  -0.02001875 -0.01607401 -0.01113183  0.01173896
  0.0241542   0.0146711   0.02720885 -0.00893924 -0.02463386  0.02043207
  0.00117172 -0.00299422 -0.03500063 -0.02490994 -0.00186201 -0.01048642
  0.00416683 -0.02354497  0.02244171 -0.00035852  0.00455083  0.00457442
  0.00671884 -0.02914382 -0.03316241 -0.01098297 -0.02663405 -0.01531737
 -0.00521354  0.02260706  0.00517672 -0.00044164 -0.02802282  0.00806365
 -0.021056   -0.00537763 -0.02120621  0.01116537  0.01538282  0.00535278
 -0.02515638 -0.01081753  0.02059971 -0.00317164 -0.01097768  0.00674097
 -0.02385465 -0.02949454 -0.01108394 -0.01966823  0.01692824  0.01644986
  0.01151329 -0.02267971  0.04476128 -0.00267644 -0

array([196369, 163204, 282357,  47707, 232777, 171281, 145746,  40097,
        19368, 166947,  59370, 278939, 244514, 257205, 178470, 108908,
       182571, 104451, 281172,  14103,  98171, 302049, 180923, 204220,
        73078])

Teste de 'sanidade', calculando similaridade de documentos em relação ao conjunto todo

In [108]:
ranks = []
second_ranks = []

for doc_id in np.random.randint(0, len(train_documents), size=1000):
    inferred_vector = model.infer_vector(train_documents[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [109]:
collections.Counter(ranks)

Counter({0: 914,
         1: 26,
         2: 4,
         3: 4,
         8: 3,
         20: 1,
         83: 1,
         7563: 1,
         22: 1,
         471: 1,
         13: 1,
         106: 1,
         1976: 1,
         2775: 1,
         2183: 1,
         7: 1,
         1698: 1,
         433: 1,
         1486: 1,
         7977: 1,
         733: 1,
         299: 1,
         12: 1,
         15: 1,
         2843: 1,
         373: 1,
         5291: 1,
         3827: 1,
         85: 1,
         505: 1,
         11699: 1,
         640: 1,
         140: 1,
         247031: 1,
         1522: 1,
         90: 1,
         336: 1,
         10882: 1,
         4: 1,
         7051: 1,
         17: 1,
         82523: 1,
         729: 1,
         229: 1,
         24111: 1,
         2300: 1,
         2553: 1,
         2766: 1,
         44: 1,
         39: 1,
         181: 1,
         123: 1,
         5: 1,
         36: 1})

91.4 % das amostras são similares a elas mesmas, o que é um bom sinal

In [112]:
doc_id = 50

inferred_vector = model.infer_vector(train_documents[doc_id].words)
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_documents[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_documents[sims[index][0]].words)))

Document (50): «sou aposentado fisso emprestimo consgnado mas ate agora nao caiu na minha conta diz que foi depositado»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/c,d400,n20,w5,mc5,s1e-05,t8>:

MOST (50, 0.7601292133331299): «sou aposentado fisso emprestimo consgnado mas ate agora nao caiu na minha conta diz que foi depositado»

SECOND-MOST (178628, 0.4619848430156708): «liberem minha portabilidadi agora»

MEDIAN (149240, 0.003771970747038722): «comprei um voo com cia aerea gol no dia exatos mes de antecedencia da data do voo cujo itinerario original era data belo horizonte confins cnf sao paulo congonhas cgh embarque partida chegada no dia menos de horas antes do voo orginal recebi um mail sobre cancelamento do voo informacao sobre novo voo marcado data belo horizonte confins cnf sao paulo congonhas cgh partida hrs chegada hrs imediatamente entrei em contato com empresa informando sobre impossibilidade de realizacao do voo na forma como alterada no mail pelos seguintes motivos comp

In [125]:
semantic_query = 'serasa serasa experian inscrita'

inferred_vector = model.infer_vector(semantic_query.split(' '))
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

print(u'%s %s: «%s»\n' % ('MOST', sims[index], ' '.join(train_documents[sims[index][0]].words)))

MOST (177767, -0.4176015257835388): «serasa serasa experian inscrita no cnpj mf sob nº xxxxx com sede na alameda dos quinimuras planalto paulista cep xxxxx considerando que nao houve qualquer consentimento de minha parte para coleta tratamento dos meus dados pessoais referentes pesquisa do meu cpf na base de dados administrada por senhoria conforme manda art da lei de ainda considerando que historico de pesquisa do meu cpf acessivel qualquer membro do sistema do serasa carece de consentimento expresso na forma do do art da lei de tenho ciencia que historico de pesquisa do meu cpf nao significa restricao ou negativacao assumo todas as consequencias do meu requerimento abaixo formulado»



Testando o modelo com o conjunto de teste

In [129]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_documents) - 1)
inferred_vector = model.infer_vector(test_documents[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_documents[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_documents[sims[index][0]].words)))

Test Document (39897): «me ofereceram fixo xxxxx por sem cobrancas adicionais porem as contas chegam acima de desde assinatura que deveria chegar chega quase valor combinado dia liguei reclamando combinaram que eu pagaria so de assinatura bina gratuita levei minha fatura na loja me mostraram que estou pagando quase reais de assinatura ainda cobrando por pacotes que nao contratei fui no procon informaram que tenho direito devolucao de todos os valores cobrados acima dos em dobro dos ultimos anos pago mensalmente por servicos que nao solicitei nao contratei ativaram plano franquia ldn de mes que nao reconheco outro de mes plano alternativo vc alem de pacote digital super mes que tambem nao reconheco nao contratei todas estas cobrancas sao indevidas ja que nao contratei estes servicos operadora ativou sem minha autorizacao liguei no atendimento da anatel atendente me disse que nesses casos legislacao determina que operadora devolva em dobro todos os valores cobrados indevidos art consumid

Salvando modelo Doc2Vec

In [130]:
model.save('../model/doc2vec_cliams')

2024-09-18 21:03:42,659 : INFO : Doc2Vec lifecycle event {'fname_or_handle': '../model/doc2vec_cliams', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-09-18T21:03:42.658065', 'gensim': '4.3.3', 'python': '3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'saving'}
2024-09-18 21:03:42,663 : INFO : storing np array 'vectors' to ../model/doc2vec_cliams.dv.vectors.npy
2024-09-18 21:03:43,972 : INFO : storing np array 'vectors' to ../model/doc2vec_cliams.wv.vectors.npy
2024-09-18 21:03:44,175 : INFO : storing np array 'syn1neg' to ../model/doc2vec_cliams.syn1neg.npy
2024-09-18 21:03:46,062 : INFO : not storing attribute cum_table
2024-09-18 21:03:46,110 : INFO : saved ../model/doc2vec_cliams
